In [3]:
# Importar las bibliotecas necesarias
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from datetime import datetime

In [4]:
data = pd.read_csv('diabetes_prediction_dataset.csv')

In [5]:
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 9 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   gender               100000 non-null  object 
 1   age                  100000 non-null  float64
 2   hypertension         100000 non-null  int64  
 3   heart_disease        100000 non-null  int64  
 4   smoking_history      100000 non-null  object 
 5   bmi                  100000 non-null  float64
 6   HbA1c_level          100000 non-null  float64
 7   blood_glucose_level  100000 non-null  int64  
 8   diabetes             100000 non-null  int64  
dtypes: float64(3), int64(4), object(2)
memory usage: 6.9+ MB


In [6]:
columnas_categoricas = data.select_dtypes(include=['object']).columns

In [7]:
for columna in columnas_categoricas:
  le = LabelEncoder()
  data[columna] = le.fit_transform(data[columna])

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 9 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   gender               100000 non-null  int64  
 1   age                  100000 non-null  float64
 2   hypertension         100000 non-null  int64  
 3   heart_disease        100000 non-null  int64  
 4   smoking_history      100000 non-null  int64  
 5   bmi                  100000 non-null  float64
 6   HbA1c_level          100000 non-null  float64
 7   blood_glucose_level  100000 non-null  int64  
 8   diabetes             100000 non-null  int64  
dtypes: float64(3), int64(6)
memory usage: 6.9 MB


In [9]:
X = data.drop('diabetes', axis=1).values #se elimina la columna diabetes se crea un nuevo dataframe
y = data['diabetes'].values #solo se escoge la columna de diabetea

In [10]:
#se separan los datos de testeo y te entrenamiento
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
# Normalizar los datos
#escala las características de tus datos utilizando la transformación de escala mínima y
#máxima, lo que resulta en características que están en el rango de 0 a 1. Esto es útil
#para asegurarse de que todas las características tengan la misma escala
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [12]:
# Convertir los datos en tensores de PyTorch para tener compatibilidad y  calculos eficientes
#se coloca los datos ya normalizados de las caracteristicas
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

In [13]:
#muestra la definición de una clase LogisticRegression que implementa un modelo
#de regresión logística
class LogisticRegression(nn.Module):
    def __init__(self, input_size):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(input_size, 1) #mapea las características de entrada al espacio de salida
        self.sigmoid = nn.Sigmoid() #define la función de activación sigmoide

    def forward(self, x):
        out = self.linear(x)
        out = self.sigmoid(out)
        return out

In [14]:
#se crea una instancia del modelo LogisticRegression
input_size = X_train.shape[1] #calcula el tamaño de las características de entrada
model = LogisticRegression(input_size)

In [15]:
# Definir la función de pérdida  y el optimizador
criterion = nn.BCELoss() #pérdida de entropía cruzada binaria (binary cross-entropy loss)
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [16]:
# Entrenar el modelo
num_epochs = 1000
for epoch in range(num_epochs):
    # Calcular la salida del modelo y la pérdida
    outputs = model(X_train)
    loss = criterion(outputs, y_train.view(-1, 1))

    # Realizar la retropropagación y el paso de optimización
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Imprimir la pérdida en cada 100 epochs
    if (epoch + 1) % 100 == 0:
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')

Epoch 100/1000, Loss: 0.5087358355522156
Epoch 200/1000, Loss: 0.42393311858177185
Epoch 300/1000, Loss: 0.3822740316390991
Epoch 400/1000, Loss: 0.3594793975353241
Epoch 500/1000, Loss: 0.3458564877510071
Epoch 600/1000, Loss: 0.3370881974697113
Epoch 700/1000, Loss: 0.33106282353401184
Epoch 800/1000, Loss: 0.3266677260398865
Epoch 900/1000, Loss: 0.3232811689376831
Epoch 1000/1000, Loss: 0.320539265871048


In [17]:
# Evaluar el modelo en el conjunto de prueba
#Se realiza una evaluación del modelo  en un conjunto de prueba y calcula la precisión (accuracy)
#de las predicciones.
with torch.no_grad():
    outputs = model(X_test)
    predicted = (outputs >= 0.5).float()
    accuracy = (predicted == y_test.view(-1, 1)).sum().item() / y_test.size(0)
    print(f'Accuracy on test set: {accuracy}') #imprime la precisión obtenida en el conjunto de prueba

Accuracy on test set: 0.9146
